# Demo's for AZ-2005

Prerequisites:

- This demo leverages Polyglot Notebooks extension in VSCode. You can get this [here](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode). This extension brings support for multi-language (C#) notebooks to Visual Studio Code.
- Make sure you also have Azure OpenAI resource created, and have deployed a gpt-4o model. Instructions can be found [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource?pivots=web-portal).
- Read the documentation for Semantic Kernel [here](https://learn.microsoft.com/en-us/semantic-kernel/).

Consider also following resources:

- https://learn.microsoft.com/en-us/semantic-kernel/get-started/detailed-samples?pivots=programming-language-csharp

## Import packages

Here are some nuget packages we need to import. Notice the Microsoft.SemanticKernel version number

In [ ]:
#r "nuget:Microsoft.Extensions.Logging, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget:Microsoft.Extensions.Logging.Debug, 8.0.0"

#r "nuget:Microsoft.SemanticKernel, 1.16.1"
#r "nuget:Microsoft.SemanticKernel.Plugins.Core, 1.16.2-alpha"
#r "nuget:dotenv.net, 3.0.0"

## Set/Get Environment variables

Make sure you have a .env file created in the root with the following 2 keys:

OPENAI_ENDPOINT=https://[your resource]-[region].openai.azure.com/

OPENAI_KEY=xxxxxxx

In [ ]:
using dotenv.net;
using dotenv.net.Utilities;

DotEnv.Load();

var openai_endpoint = EnvReader.GetStringValue("OPENAI_ENDPOINT");
var openai_key = EnvReader.GetStringValue("OPENAI_KEY");

## Create the Semantic Kernel

The kernel is at the center of your agents. 

Because the kernel has all of the services and plugins necessary to run both native code and AI services, it is used by nearly every component within the Semantic Kernel SDK to power your agents. This means that if you run any prompt or code in Semantic Kernel, the kernel will always be available to retrieve the necessary services and plugins.

When you invoke a prompt from the kernel. When you do so, the kernel will...

1. Select the best AI service to run the prompt.
2. Build the prompt using the provided prompt template.
3. Send the prompt to the AI service.
4. Receive and parse the response.
5. And finally return the response from the LLM to your application.

The Semantic Kernel SDK supports HuggingFace, OpenAI, and Azure OpenAI LLMs. For this example, we use Azure OpenAI. Make sure you have deployed an Azure OpenAI Resource, deployed a model gpt-4o and configured the environment variables

Notice below we also add logging as a service to the kernel.

In [ ]:
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.Logging.Console;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Plugins.Core;

var builder = Microsoft.SemanticKernel.Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(
         "gpt-4o",         // Azure OpenAI Deployment Name. Make sure this is correct
         openai_endpoint,  // Azure OpenAI Endpoint
         openai_key);      // Azure OpenAI Key

builder.Services.AddLogging(c => c.AddDebug()
                                  .SetMinimumLevel(LogLevel.Trace) // don't do this in production!
                                  .AddConsole());
      

var kernel = builder.Build();

# Scenario 0: Test that your kernel and endpoint is working

In [ ]:
string prompt = "Give me a list of breakfast foods with eggs and cheese";

var result = await kernel.InvokePromptAsync(prompt);

result.ToString().DisplayAs("text/markdown");
result

## Scenario 1: Running your first prompt with Semantic Kernel

In this first scenario, we try to get an answer from this question. Notice that the LLM does not know anything about my solar panels and instead, gives a very generic answer.

In [ ]:
string request = "I want to know how much power my solar panels are providing.";
string prompt = $"What is the intent of this request? {request}";

var result = await kernel.InvokePromptAsync(prompt);

result.ToString().DisplayAs("text/markdown");

## Scenario 2: Running your first prompt with Semantic Kernel - set Temperature and MaxTokens

We can control the answer a bit, by specifying Temperate and MaxTokens

- **Temperature**: The sampling temperature, between 0 and 1. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. If set to 0, the model will use log probability to automatically increase the temperature until certain thresholds are hit.

- **MaxTokens**: The maximum number of tokens that can be generated in the chat completion. The total length of input tokens and generated tokens is limited by the model's context length.

In [ ]:
string request = "I want to know how much power my solar panels are providing.";
string prompt = $"What is the intent of this request? {request}";

var result = await kernel.InvokePromptAsync(prompt, new(new OpenAIPromptExecutionSettings()
{
    MaxTokens = 10,
    Temperature = 0.7
}));

result.ToString().DisplayAs("text/markdown");
result.Display();

## Scenario 3: Improving the prompt with prompt engineering

Prompt engineering is crucial because it directly impacts the effectiveness and reliability of the LLM. The goal is to frame questions or tasks in such a way that the AI provides the most accurate, relevant, and contextually appropriate **output**.

In [ ]:
string request = "I want to know how much power my solar panels are providing.";

string prompt = @$"
    What is the intent of this request? 
    {request}
    You can choose between GetSolarEnergyToday, GetSolarPower, GetSolarBatteryPercentage, StartChargingCar.";

var result = await kernel.InvokePromptAsync(prompt);

result.ToString().DisplayAs("text/markdown");

## Scenario 4: Add structure to the output with formatting

Here, we go even one step further and make the output even more predictable.

In [ ]:
string request = "I want to know how much power my solar panels are providing.";

string prompt = prompt = @$"
    Instructions: What is the intent of this request?
    Choices: GetSolarEnergyToday, GetSolarPower, GetSolarBatteryPercentage, StartChargingCar.
    User Input: {request}
    Intent: ";

var result = await kernel.InvokePromptAsync(prompt);

result.ToString().DisplayAs("text/markdown");

In [ ]:
string prompt = @"
Instructions: Identify the from and to destinations and dates from the user's request
>User: Can you give me a list of flights from Seattle to Tokyo? I want to travel from March 11 to March 18.
>Output: Seattle|Tokyo|03/11/2024|03/18/2024

>User: I have a vacation from June 1 to July 22. I want to go to Greece. I live in Chicago.
>Output: ";

var result = await kernel.InvokePromptAsync(prompt);

result.ToString().DisplayAs("text/markdown");

## Scenario 5: Even more control over the output as JSON

In [ ]:
string request = "I want to know how much power my solar panels are providing.";

string prompt = $$"""
    ## Instructions
    Provide the intent of the request using the following format:

    ```json
    {
        "intent": {intent}
    }
    ```

    ## Choices
    You can choose between the following intents:

    ```json
    ["GetSolarEnergyToday", "GetSolarPower", "GetSolarBatteryPercentage", "StartChargingCar"]
    ```

    ## User Input
    The user input is:

    ```json
    {
        "request": "{{request}}"
    }
    ```

    ## Intent
""";

var result = await kernel.InvokePromptAsync(prompt);

result.ToString().DisplayAs("text/markdown");

## Scenario 6: Provide examples with few-shot prompting

Few-shot prompting is a technique in AI where a model is given a small number of examples (usually a few) along with the prompt to help it understand the desired output format or context.

In [ ]:
string request = "I want to know how much power my solar panels are providing.";

string prompt = @$"
    Instructions: What is the intent of this request?
    If you don't know the intent, don't guess; instead respond with ""Unknown"".
    Choices: GetSolarEnergyToday, GetSolarPower, GetSolarBatteryPercentage, StartChargingCar.

    User Input: How much energy did my solar panels provide today?
    Intent: GetSolarEnergyToday

    User Input: Can you start charging my car?
    Intent: StartChargingCar

    User Input: {request}
    Intent: ";

var result = await kernel.InvokePromptAsync(prompt);

result.ToString().DisplayAs("text/markdown");

## Scenario 7: Chatbot (async)

With chat completion, you can simulate a back-and-forth conversation with an AI agent.

**Import note**: If you ask follow-up questions, the model does not remember those.

Tip: In VSCode, when you run the cell below, it will ask you for input in the command palette (TOP of the window). You can enter following question:

"who was JF Kennedy?" and a follow up question "when did he die?" (observe the answer)

There are two main ways to use a chat completion service:

- **Non-streaming**: You wait for the service to generate an entire response before returning it to the user.
- **Streaming**: Individual chunks of the response are generated and returned to the user as they are created. (as you can observe in this example)

In [ ]:
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

while (true)
{
    Console.Write("User > ");

    var request = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter your message (type EXIT to quit):");

    if (request.ToLower() == "exit" || String.IsNullOrEmpty(request))
    {
        break;
    }

    Console.Write(request);
    Console.WriteLine();

    var result = chatCompletionService.GetStreamingChatMessageContentsAsync(request, kernel: kernel);

    string fullMessage = "";
    Console.Write("Assistant > ");

    await foreach (var content in result)
    {
        Console.Write(content.Content);
        fullMessage += content.Content;
    }

    Console.WriteLine();
    Console.WriteLine();
}

## Scenario 8: Chatbot (async, with history)

The chat history object is used to maintain a record of messages in a chat session. It is used to store messages from different authors, such as users, assistants, tools, or the system. As the primary mechanism for sending and receiving messages, the chat history object is essential for maintaining context and continuity in a conversation.

In [ ]:
ChatHistory history = [];   // add chatHistory

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

while (true)
{
    Console.Write("User > ");

    var request = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter your message (type EXIT to quit):");

    if (request.ToLower() == "exit" || String.IsNullOrEmpty(request))
    {
        break;
    }

    history.AddUserMessage(request!);       // add user message to history  

    Console.Write(request);
    Console.WriteLine();

    var result = chatCompletionService.GetStreamingChatMessageContentsAsync(history, kernel: kernel);  // replace request with history

    string fullMessage = "";
    Console.Write("Assistant > ");

    await foreach (var content in result)
    {
        Console.Write(content.Content);
        fullMessage += content.Content;
    }

    history.AddAssistantMessage(fullMessage);  // add assistant message to history

    Console.WriteLine();
    Console.WriteLine();
}

## Scenario 9: Chatbot (async, with history and a persona)

Often called a "meta prompt" or "instruction", the persona is a prompt that is used to influence how the agent responds to stimuli. This allows you to influence how your agents plan tasks, generate responses, and interact with users.

In [ ]:
ChatHistory history = [];   // add chatHistory

history.AddSystemMessage($"You should answer as a 10-year old child. In addition, greet the user by his name: {Environment.UserName}"); // add a persona

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

while (true)
{
    Console.Write("User > ");

    var request = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter your message (type EXIT to quit):");

    if (request.ToLower() == "exit" || String.IsNullOrEmpty(request))
    {
        break;
    }

    history.AddUserMessage(request!);       // add user message to history  

    Console.Write(request);
    Console.WriteLine();

    var result = chatCompletionService.GetStreamingChatMessageContentsAsync(history, kernel: kernel);  // replace request with history

    string fullMessage = "";
    Console.Write("Assistant > ");

    await foreach (var content in result)
    {
        Console.Write(content.Content);
        fullMessage += content.Content;
    }

    history.AddAssistantMessage(fullMessage);  // add assistant message to history

    Console.WriteLine();
    Console.WriteLine();
}

## Scenario 10: Plugins from Microsoft.SemanticKernel.Plugins.Core (1.16.2-alpha)

Plugins are a key component of Semantic Kernel. If you have already used plugins from ChatGPT or Copilot extensions in Microsoft 365, you’re already familiar with them. With plugins, you can encapsulate your existing APIs into a collection that can be used by an AI. This allows you to give your AI the ability to perform actions that it wouldn’t be able to do otherwise.

Behind the scenes, Semantic Kernel leverages function calling, a native feature of most of the latest LLMs to allow LLMs, to perform planning and to invoke your APIs. With function calling, LLMs can request (i.e., call) a particular function. Semantic Kernel then marshals the request to the appropriate function in your codebase and returns the results back to the LLM so the LLM can generate a final response.

Following plugins are available in this session:
- ConversationSummaryPlugin
- FileIOPlugin
- HttpPlugin
- MathPlugin
- TextPlugin
- TimePlugin
- WaitPlugin

In [ ]:
#pragma warning disable SKEXP0050 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

KernelPlugin conversationSummaryPlugin = kernel.Plugins.AddFromType<ConversationSummaryPlugin>();
KernelPlugin fileIOPlugin = kernel.Plugins.AddFromType<FileIOPlugin>();
KernelPlugin httpPlugin = kernel.Plugins.AddFromType<HttpPlugin>();
KernelPlugin mathPlugin = kernel.Plugins.AddFromType<MathPlugin>();
KernelPlugin textPlugin = kernel.Plugins.AddFromType<TextPlugin>();
KernelPlugin timePlugin = kernel.Plugins.AddFromType<TimePlugin>();
KernelPlugin waitPlugin = kernel.Plugins.AddFromType<WaitPlugin>();

#pragma warning restore SKEXP0050 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

In [ ]:
var executionSettings = new OpenAIPromptExecutionSettings
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

In [ ]:
private const string ChatTranscript =
        @"
John: Hello, how are you?
Jane: I'm fine, thanks. How are you?
John: I'm doing well, writing some example code.
Jane: That's great! I'm writing some example code too.
John: What are you writing?
Jane: I'm writing a chatbot.
John: That's cool. I'm writing a chatbot too.
Jane: What language are you writing it in?
John: I'm writing it in C#.
Jane: I'm writing it in Python.
John: That's cool. I need to learn Python.
Jane: I need to learn C#.
John: Can I try out your chatbot?
Jane: Sure, here's the link.
John: Thanks!
Jane: You're welcome.
Jane: Look at this poem my chatbot wrote:
Jane: Roses are red
Jane: Violets are blue
Jane: I'm writing a chatbot
Jane: What about you?
John: That's cool. Let me see if mine will write a poem, too.
John: Here's a poem my chatbot wrote:
John: The singularity of the universe is a mystery.
John: The universe is a mystery.
John: The universe is a mystery.
John: The universe is a mystery.
John: Looks like I need to improve mine, oh well.
Jane: You might want to try using a different model.
Jane: I'm using the GPT-3 model.
John: I'm using the GPT-2 model. That makes sense.
John: Here is a new poem after updating the model.
John: The universe is a mystery.
John: The universe is a mystery.
John: The universe is a mystery.
John: Yikes, it's really stuck isn't it. Would you help me debug my code?
Jane: Sure, what's the problem?
John: I'm not sure. I think it's a bug in the code.
Jane: I'll take a look.
Jane: I think I found the problem.
Jane: It looks like you're not passing the right parameters to the model.
John: Thanks for the help!
Jane: I'm now writing a bot to summarize conversations. I want to make sure it works when the conversation is long.
John: So you need to keep talking with me to generate a long conversation?
Jane: Yes, that's right.
John: Ok, I'll keep talking. What should we talk about?
Jane: I don't know, what do you want to talk about?
John: I don't know, it's nice how CoPilot is doing most of the talking for us. But it definitely gets stuck sometimes.
Jane: I agree, it's nice that CoPilot is doing most of the talking for us.
Jane: But it definitely gets stuck sometimes.
John: Do you know how long it needs to be?
Jane: I think the max length is 1024 tokens. Which is approximately 1024*4= 4096 characters.
John: That's a lot of characters.
Jane: Yes, it is.
John: I'm not sure how much longer I can keep talking.
Jane: I think we're almost there. Let me check.
Jane: I have some bad news, we're only half way there.
John: Oh no, I'm not sure I can keep going. I'm getting tired.
Jane: I'm getting tired too.
John: Maybe there is a large piece of text we can use to generate a long conversation.
Jane: That's a good idea. Let me see if I can find one. Maybe Lorem Ipsum?
John: Yeah, that's a good idea.
Jane: I found a Lorem Ipsum generator.
Jane: Here's a 4096 character Lorem Ipsum text:
Jane: Lorem ipsum dolor sit amet, con
Jane: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed euismod, nunc sit amet aliquam
Jane: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed euismod, nunc sit amet aliquam
Jane: Darn, it's just repeating stuff now.
John: I think we're done.
Jane: We're not though! We need like 1500 more characters.
John: Oh Cananda, our home and native land.
Jane: True patriot love in all thy sons command.
John: With glowing hearts we see thee rise.
Jane: The True North strong and free.
John: From far and wide, O Canada, we stand on guard for thee.
Jane: God keep our land glorious and free.
John: O Canada, we stand on guard for thee.
Jane: O Canada, we stand on guard for thee.
Jane: That was fun, thank you. Let me check now.
Jane: I think we need about 600 more characters.
John: Oh say can you see?
Jane: By the dawn's early light.
John: What so proudly we hailed.
Jane: At the twilight's last gleaming.
John: Whose broad stripes and bright stars.
Jane: Through the perilous fight.
John: O'er the ramparts we watched.
Jane: Were so gallantly streaming.
John: And the rockets' red glare.
Jane: The bombs bursting in air.
John: Gave proof through the night.
Jane: That our flag was still there.
John: Oh say does that star-spangled banner yet wave.
Jane: O'er the land of the free.
John: And the home of the brave.
Jane: Are you a Seattle Kraken Fan?
John: Yes, I am. I love going to the games.
Jane: I'm a Seattle Kraken Fan too. Who is your favorite player?
John: I like watching all the players, but I think my favorite is Matty Beniers.
Jane: Yeah, he's a great player. I like watching him too. I also like watching Jaden Schwartz.
John: Adam Larsson is another good one. The big cat!
Jane: WE MADE IT! It's long enough. Thank you!
John: You're welcome. I'm glad we could help. Goodbye!
Jane: Goodbye!
";
string prompt = $"Summarize the following content: {ChatTranscript}";

var result = await kernel.InvokePromptAsync(prompt, new(executionSettings));

result.ToString().DisplayAs("text/markdown");

In [ ]:
string prompt = "What is the current time?";

var result = await kernel.InvokePromptAsync(prompt, new(executionSettings));

result.ToString().DisplayAs("text/markdown");

In [ ]:
string prompt = "Can you summarize me the following article? https://en.wikipedia.org/wiki/Triglav_Lakes_Valley";

var result = await kernel.InvokePromptAsync(prompt, new(executionSettings));

result.ToString().DisplayAs("text/markdown");

## Scenario 11: Writing your own Date and Time plugin

The easiest way to provide an AI agent with capabilities that are not natively supported is to wrap native code into a plugin. This allows you to leverage your existing skills as an app developer to extend the capabilities of your AI agents.

Behind the scenes, Semantic Kernel will then use the descriptions you provide, along with reflection, to semantically describe the plugin to the AI agent. This allows the AI agent to understand the capabilities of the plugin and how to interact with it.

When authoring a plugin, you need to provide the AI agent with the right information to understand the capabilities of the plugin and its functions. This includes:

- The name of the plugin
- The names of the functions
- The descriptions of the functions
- The parameters of the functions
- The schema of the parameters

The value of Semantic Kernel is that it can automatically generate most of this information from the code itself. As a developer, this just means that you must provide the semantic descriptions of the functions and parameters so the AI agent can understand them. If you properly comment and annotate your code, however, you likely already have this information on hand.

In [ ]:
using System.ComponentModel;

public class MyOwnDateAndTimePlugin {
    [KernelFunction]
    [Description("Gets the current time.")]
    public TimeSpan GetTime()
    {
        return TimeProvider.System.GetLocalNow().TimeOfDay;
    }

    [KernelFunction]
    [Description("Gets the current date.")]
    public DateTime GetDate()
    {
        return TimeProvider.System.GetLocalNow().Date;
    }
}

KernelPlugin myOwnDateAndTimePlugin = kernel.Plugins.AddFromType<MyOwnDateAndTimePlugin>();  // need to add the plugin

In [ ]:
string prompt = "What is the current time?";

var result = await kernel.InvokePromptAsync(prompt, new(executionSettings));

result.ToString().DisplayAs("text/markdown");

## Scenario 12: P1 meter power consumption plugin

This is a personal example that leverages a P1 meter like this https://www.homewizard.com/p1-meter/. A P1 meter refers to a smart electricity meter used primarily in Belgium and the Netherlands that includes a P1 port, which is a standardized interface. This port allows consumers to access real-time and historical energy consumption data from the smart meter. 

![image of p1 meter](https://cdn.homewizard.com/wp-content/uploads/2021/11/P1_meter_front-500x500.png)

Make sure you have a .env file in the root with following keys:

HOMEWIZARD_USERNAME=

HOMEWIZARD_PASSWORD=

HOMEWIZARD_DONGLEID=

In [ ]:
using System.Net.Http.Headers;
using System.Net.Http;
using System.Text;
using System.Text.Json;

public class P1Meter {
    private static readonly HttpClient httpClient = new HttpClient();

    [KernelFunction]
    [Description("Gets the current power consumption from the P1 homewizard energy reader")]
    public async Task<int> GetPower()
    {
        string token = await GetAuthTokenAsync();
        int powerConsumption = await GetPowerConsumptionAsync(token);
        return powerConsumption;
    }

    private async Task<string> GetAuthTokenAsync()
    {
        string url = "https://api.homewizardeasyonline.com/v1/auth/login";

        string username = Environment.GetEnvironmentVariable("HOMEWIZARD_USERNAME");
        string password = Environment.GetEnvironmentVariable("HOMEWIZARD_PASSWORD");
        string authorizationHeader = System.Convert.ToBase64String(System.Text.Encoding.UTF8.GetBytes($"{username}:{password}"));

        httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Basic", authorizationHeader);

        HttpResponseMessage response = await httpClient.GetAsync(url);
        response.EnsureSuccessStatusCode();

        string responseBody = await response.Content.ReadAsStringAsync();
        using JsonDocument doc = JsonDocument.Parse(responseBody);
        string token = doc.RootElement.GetProperty("token").GetString();

        return token;
    }

    private async Task<int> GetPowerConsumptionAsync(string token)
    {
        string dongleID = Environment.GetEnvironmentVariable("HOMEWIZARD_DONGLEID");
        string url = "https://tsdb-reader.homewizard.com/devices/date/recent";
        string payload = $@"{{
            ""devices"": [""{dongleID}""],
            ""type"": ""main_connection"",
            ""values"": true,
            ""wattage"": true,
            ""gb"": ""1s"",
            ""fill"": ""previous""
        }}";

        httpClient.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", token);
        httpClient.DefaultRequestHeaders.Accept.Add(new MediaTypeWithQualityHeaderValue("application/json"));

        HttpContent content = new StringContent(payload, Encoding.UTF8, "application/json");

        HttpResponseMessage response = await httpClient.PostAsync(url, content);
        response.EnsureSuccessStatusCode();

        string responseBody = await response.Content.ReadAsStringAsync();
        using JsonDocument doc = JsonDocument.Parse(responseBody);
        int powerConsumption = doc.RootElement.GetProperty("values")[1].GetProperty("wattage").GetInt32();

        return powerConsumption;
    }
}

KernelPlugin P1MeterPlugin = kernel.Plugins.AddFromType<P1Meter>();  // need to add the plugin

In [ ]:
string prompt = "What is my current power consumption?";

var result = await kernel.InvokePromptAsync(prompt, new(executionSettings));

result.ToString().DisplayAs("text/markdown");